In [255]:
from requests import request
import requests

def volume(x, y, z): return x*y*z
def density(mass, volume): return mass/volume

compartments = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/compartment").json()
containers = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/container").json()
shipments = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/shipment").json()
luggage = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/luggage").json()

for container in containers:
    container["volume"]  = volume(container["length"], container["width"], container["height"])

compartment_conf = [
    {"PMC":0,"PAG":0,"AKE":10},
    {"PMC":0,"PAG":0,"AKE":10},
    {"PMC":0,"PAG":0,"AKE":8},
    {"PMC":0,"PAG":0,"AKE":8},
    {"PMC":0,"PAG":0,"AKE":1},
]

no_of_first_class_luggage = luggage["nbFirstClassLuggage"]
no_of_second_class_luggage = + luggage["nbLuggage"]
avg_wt = luggage["avgWeight"] 

for shipment in shipments:
    shipment["volume"]  = volume(shipment["length"], shipment["width"], shipment["height"])
    shipment["density"]  = density(shipment["weight"], shipment["volume"])

shipments = sorted(shipments, key=lambda k: k['density'], reverse=True)

In [256]:
def knapsack(n, weight, count, values, weights):
    dp = [[[0] * (weight + 1) for _ in range(n + 1)] for _ in range(count + 1)]
    for z in range(1, count + 1):
        for y in range(1, n + 1):
            for x in range(weight + 1):
                if weights[y - 1] <= x:
                    dp[z][y][x] = max(dp[z][y - 1][x],
                                      dp[z - 1][y - 1][x - weights[y - 1]] + values[y - 1])
                else:
                    dp[z][y][x] = dp[z][y - 1][x]

    return dp[-1][-1][-1]

In [257]:
result = [
    {
        "compartmentId": 1,
        "containersWithShipments": [],
        "containersWithLuggage": []
    },
    {
        "compartmentId": 2,
        "containersWithShipments": [],
        "containersWithLuggage": []
    },
    {
        "compartmentId": 3,
        "containersWithShipments": []
    },
    {
        "compartmentId": 4,
        "containersWithShipments": []
    },
    {
        "compartmentId": 5,
        "containersWithLuggage": []
    }
]

In [258]:
def place_luggage(no_of_luggage):
    available_luggage = no_of_luggage
    print(available_luggage)
    for i in [2,1,3,4]:
        available_ake = compartment_conf [i]["AKE"]
        available_pmc = compartment_conf [i]["PMC"]
        available_pag = compartment_conf [i]["PAG"]
        available_wt = compartments[i - 1]["maxWeight"]

        for j in range(available_ake):
            compartments[i - 1]["maxWeight"] = available_wt
            if(available_luggage<=0 or available_wt<=0): break
            luggage_to_add = 38 if available_luggage >= 38 else available_luggage
            #Wt constraint
            wt_to_add = luggage_to_add* 20
            if available_wt < wt_to_add:
                luggage_to_add = available_wt//20
                wt_to_add = luggage_to_add*20
            
            compartment_conf [i-1]["AKE"] -= 1 

            available_luggage -= luggage_to_add
            available_wt -= wt_to_add
            result[i-1]["containersWithLuggage"].append({
                "containerType": "AKE",
                "nbOfLuggage": luggage_to_add
            }
            )

#firstclass
place_luggage(no_of_first_class_luggage+no_of_second_class_luggage)

#Secondclass
# place_luggage(no_of_second_class_luggage)

359


In [259]:
# for i in range(1000):
#     r
# es = requests.post(url, json = myobj)
#     print(res.status_code)
result

[{'compartmentId': 1,
  'containersWithShipments': [],
  'containersWithLuggage': [{'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 17}]},
 {'compartmentId': 2,
  'containersWithShipments': [],
  'containersWithLuggage': [{'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38}]},
 {'compartmentId': 3, 'containersWithShipments': []},
 {'compartmentId': 4, 'containersWithShipments': []},
 {'compartmentId': 5, 'containersWithLuggage': []}]

In [260]:
compartment_conf

[{'PMC': 0, 'PAG': 0, 'AKE': 8},
 {'PMC': 0, 'PAG': 0, 'AKE': 2},
 {'PMC': 0, 'PAG': 0, 'AKE': 8},
 {'PMC': 0, 'PAG': 0, 'AKE': 8},
 {'PMC': 0, 'PAG': 0, 'AKE': 1}]

In [261]:
def place_shipment():
    for i in [4, 3,2,1]:
        available_ake = compartment_conf[i]["AKE"]
        available_pmc = compartment_conf[i]["PMC"]
        available_pag = compartment_conf[i]["PAG"]
        available_wt = compartments[i - 1]["maxWeight"]

        for j in range(available_ake):
            if(len(shipments)<=0): break
            remaining_volume = containers[2]["volume"]*0.89
            if available_wt < shipments[0]["weight"]:
                break

            shipments_to_add = []
            remaining_container_wt = 1588

            while (True):
                if(len(shipments)<=0): break
                if available_wt < shipments[0]["weight"]: break
                
                print(len(shipments))

                if remaining_volume >= shipments[0]["volume"] :
                    if(len(shipments)<=0): break
                    if available_wt < shipments[0]["weight"] or remaining_container_wt < shipments[0]["weight"]:
                        break


                    remaining_container_wt = remaining_container_wt - shipments[0]["weight"]
                    available_wt = available_wt - shipments[0]["weight"]
                    remaining_volume = remaining_volume - shipments[0]["volume"]
                    shipment = shipments.pop(0)
                    shipments_to_add.append(shipment["id"])

                else:
                    break
            if (len(shipments_to_add) > 0):
                result[i-1]["containersWithShipments"].append({
                    "containerType": "AKE",
                    "shipments": shipments_to_add
                })


place_shipment()


98
97
96
95
94
94
93
92
91
90
90
89
88
87
86
86
85
84
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83
83


In [262]:
import json

json.dumps(result)

'[{"compartmentId": 1, "containersWithShipments": [], "containersWithLuggage": [{"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 17}]}, {"compartmentId": 2, "containersWithShipments": [], "containersWithLuggage": [{"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}]}, {"compartmentId": 3, "containersWithShipments": [{"containerType": "AKE", "shipments": [87, 88, 89, 90]}, {"containerType": "AKE", "shipments": [91, 92, 93, 94]}, {"containerType": "AKE", "shipments": [95, 96, 97]}]}, {"compartmentId": 4, "containersWithShipments": [{"containerType": "AKE", "shipments": [83, 84, 85, 86]}]}, {"compartmentId": 5, "containersWithLuggage": []}]'

In [263]:
shipments

[{'id': 39,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 40,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 41,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 42,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 43,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 44,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 45,
  'awb': '074-51